In [1]:
import glob
import time
from sokoban import Warehouse

import multiprocessing as mp
import multiprocessing.queues as mpq

from typing import Tuple, Callable, Dict

import json

## This Testing File

This file shows how to automate testing across all warehouses **with a timeout option** for warehouses that take longer than 3 minutes. It may assist you in performing a more widescale evaluation.

In [2]:
#Import the mySokobanSolver.py file to import all functions
from mySokobanSolver import *

## Visualise results

In [3]:
def test_warehouse(problem_file, macro = False):
    '''
    This function will test the performance of your warehouse for either macro or elem solutions and return the result.
    You can check if this solution works with your gui, or by cleverly using the check_action_seq function.
    '''

    wh = Warehouse()
    wh.load_warehouse(problem_file)

    if macro:
        student_answer =  solve_sokoban_macro(wh)
    else:
        student_answer = solve_sokoban_elem(wh)
        
    return student_answer
#.............................................................


In [ ]:
all_warehouses = sorted(glob.glob('warehouses/*.txt'))

elem_step = {}
macro_step = {}

elem_time = {}
macro_time = {}

def warehouse_timeout(args: Tuple[object], q: mp.Queue):
    #Do not alter this code.
    q.put(test_warehouse(*args))
    
def test_with_timeout(problem_file, macro = False, timeout = 180):
    """
    This function tests on a warehouse with the ability to timeout after a specified number of seconds. 

    Parameters:
    problem_file (str): directory of a warehouse
    macro (bool): indicates whether to use the macro solver. If false, will use the elem solver
    timeout (int): The number of seconds the solver can run without timing out.

    Returns:
    The solver solution or the string "Timed out"
    """
    
    global elem_step, macro_step
    
    q_worker = mp.Queue()
    proc = mp.Process(target=warehouse_timeout, args=((problem_file,macro), q_worker))
    proc.start()
    try:
        res = q_worker.get(timeout=timeout)
    except mpq.Empty:
        proc.terminate()
        res = "Timed out"

    if macro == True:
        macro_step[problem_file] = res
    else:
        elem_step[problem_file] = res
            
    return res

def save_to_file (my_dict, filename):
     with open(filename, 'w') as file:
        json.dump(my_dict, file)

for problem_file in all_warehouses:
    print(f'Testing {problem_file}')
    
    print("---Testing Elem Solver---")
    s = time.time()
    a = test_with_timeout(problem_file)
    print(f'Answer: {a}')
    taken_time = time.time()-s
    print(f'Time taken: {taken_time :.3f} seconds')
    # print(f'[Total elem moves]: {elem_path_total}')
    # print(f'[Total elem solves]: {elem_solve_total}')

    elem_time[problem_file] = taken_time*1000
    save_to_file(elem_time, 'elem_time.json')
    save_to_file(elem_step, 'elem_step.json')
    
    print("---Testing Macro Solver---")
    s = time.time()
    a = test_with_timeout(problem_file, True)
    print(f'Answer for Macro Solver: {a}')
    taken_time = time.time()-s
    print(f'Time taken: {taken_time :.3f} seconds')
    # print(f'[Total macro moves]: {macro_path_total}')   
    # print(f'[Total macro solves]: {macro_solve_total}')   
    
    macro_time[problem_file] = taken_time *1000
    save_to_file(macro_time, 'macro_time.json')
    save_to_file(macro_step, 'macro_step.json')
    print('==============================================================================================')